In [10]:
import os
from typing import List, Dict, Any
import pandas as pd

# Set up 

In [14]:
from langchain_core.documents import Document
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter,
    CharacterTextSplitter,  
    TokenTextSplitter,
)

## Understamding Document Structure in langchain

In [14]:
doc = Document(
    page_content="This is a sample document. It contains text data for processing.",
    metadata={
        "source": "sample.txt",
        "author": "John Doe",
        "length": 42,
        "page": 1,
        "keywords": ["sample", "document", "text"],
        "custom_info": {"reviewed": True, "rating": 4.5}
    }
)

## Understamding Document Structure in langchain
print(doc.page_content)
print(doc.metadata)


This is a sample document. It contains text data for processing.
{'source': 'sample.txt', 'author': 'John Doe', 'length': 42, 'page': 1, 'keywords': ['sample', 'document', 'text'], 'custom_info': {'reviewed': True, 'rating': 4.5}}


# Text Files (.txt) - The Simplest Case 

In [3]:
## Create a simple text file
import os
os.makedirs("data/text_files", exist_ok = True)

In [4]:
sample_text  = {
    "data/text_files/sample1.txt":"""
    Python is a high-level, interpreted programming language known for its simplicity and readability.
Created by Guido van Rossum and first released in 1991, Python has become one of the most popular
programming languages in the world.

Key Features:
- Easy to learn and use
- Extensive standard library
- Cross-platform compatibility
- Strong community support

Python is widely used in web development, data science, artificial intelligence, and automation.""",
    
    "data/text_files/machine_learning.txt": """Machine Learning Basics

Machine learning is a subset of artificial intelligence that enables systems to learn and improve
from experience without being explicitly programmed. It focuses on developing computer programs
that can access data and use it to learn for themselves.

Types of Machine Learning:
1. Supervised Learning: Learning with labeled data
2. Unsupervised Learning: Finding patterns in unlabeled data
3. Reinforcement Learning: Learning through rewards and penalties

Applications include image recognition, speech processing, and recommendation systems
    
    
"""
}



In [13]:
for filepath, content in sample_text.items():
    with open(filepath, "w", encoding = "utf-8") as f:
        f.write(content)

### TextLoader -Read Single File

In [5]:
from langchain.document_loaders import TextLoader
from langchain_community.document_loaders import TextLoader

## Loading a single text file
loader = TextLoader("data/text_files/sample1.txt", encoding = "utf-8")
documents = loader.load()
print(f"📄 Load {len(documents)} document")
print(f"Content preview {documents[0].page_content[:100]}")
print(f"Metadata : {documents[0].metadata}")

📄 Load 1 document
Content preview 
    Python is a high-level, interpreted programming language known for its simplicity and readabili
Metadata : {'source': 'data/text_files/sample1.txt'}


# DirectoryLoader - Multiple Text Files

In [16]:
from langchain_community.document_loaders import DirectoryLoader

## Load all the text files from the directory

dir_load = DirectoryLoader(
    "data/text_files",
    glob = "**/*.txt",
    loader_cls = TextLoader,# load class to use
    loader_kwargs = {'encoding': "utf-8"},
    show_progress=True
)

docm = dir_load.load()
print(f"Loaded {len(docm)} documents")
for i, doc in enumerate(docm):
    print(f"\n Document {i+1}")
    print(f"  Sourse: {doc.metadata["source"]}")
    print(f"  Length: {len(doc.page_content)} characters")
    

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 979.29it/s]

Loaded 2 documents

 Document 1
  Sourse: data\text_files\machine_learning.txt
  Length: 571 characters

 Document 2
  Sourse: data\text_files\sample1.txt
  Length: 461 characters


# 📊 Analysis
print("\n📊 DirectoryLoader Characteristics:")
print("✅ Advantages:")
print("  - Loads multiple files at once")
print("  - Supports glob patterns")
print("  - Progress tracking")
print("  - Recursive directory scanning")

print("\n❌ Disadvantages:")
print("  - All files must be same type")
print("  - Limited error handling per file")
print("  - Can be memory intensive for large directories")

## Text Splitting Statergies

In [23]:
## Different text splitter strategies

from langchain.text_splitter import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
    TokenTextSplitter
)

In [25]:
docm

[Document(metadata={'source': 'data\\text_files\\machine_learning.txt'}, page_content='Machine Learning Basics\n\nMachine learning is a subset of artificial intelligence that enables systems to learn and improve\nfrom experience without being explicitly programmed. It focuses on developing computer programs\nthat can access data and use it to learn for themselves.\n\nTypes of Machine Learning:\n1. Supervised Learning: Learning with labeled data\n2. Unsupervised Learning: Finding patterns in unlabeled data\n3. Reinforcement Learning: Learning through rewards and penalties\n\nApplications include image recognition, speech processing, and recommendation systems\n\n\n'),
 Document(metadata={'source': 'data\\text_files\\sample1.txt'}, page_content='\n    Python is a high-level, interpreted programming language known for its simplicity and readability.\nCreated by Guido van Rossum and first released in 1991, Python has become one of the most popular\nprogramming languages in the world.\n\nKe

## Method 1 - Character Text Splitter

In [27]:
text = docm[0].page_content
text

'Machine Learning Basics\n\nMachine learning is a subset of artificial intelligence that enables systems to learn and improve\nfrom experience without being explicitly programmed. It focuses on developing computer programs\nthat can access data and use it to learn for themselves.\n\nTypes of Machine Learning:\n1. Supervised Learning: Learning with labeled data\n2. Unsupervised Learning: Finding patterns in unlabeled data\n3. Reinforcement Learning: Learning through rewards and penalties\n\nApplications include image recognition, speech processing, and recommendation systems\n\n\n'

In [62]:
character_text_splitter = CharacterTextSplitter(
    separator=" ",
    chunk_size = 200,
    chunk_overlap = 20,      # overlab between chunks
    length_function = len
)

char_doc = character_text_splitter.split_text(text)
print(f" Create {len(char_doc)} chunks")
print(f"First chunk : {char_doc[0][:200]}")
print(f"-------------------------------")
print(f"Second chunk : {char_doc[1][:200]}")
print(f"-------------------------------")
print(f"Third chunk : {char_doc[2][:200]}")



 Create 3 chunks
First chunk : Machine Learning Basics

Machine learning is a subset of artificial intelligence that enables systems to learn and improve
from experience without being explicitly programmed. It focuses on developing
-------------------------------
Second chunk : on developing computer programs
that can access data and use it to learn for themselves.

Types of Machine Learning:
1. Supervised Learning: Learning with labeled data
2. Unsupervised Learning:
-------------------------------
Third chunk : Learning: Finding patterns in unlabeled data
3. Reinforcement Learning: Learning through rewards and penalties

Applications include image recognition, speech processing, and recommendation systems


# Method 2:  Recursive Character TextSplitter

In [70]:
recursive_splitter = RecursiveCharacterTextSplitter(
    separators= [" "],
    chunk_size = 200,
    chunk_overlap = 20,
    length_function = len
)

recurv_splitter = recursive_splitter.split_text(text)


print(f"Create {len(recurv_splitter)} chunks")
print(f"First chunk : {recurv_splitter[0][:200]}")
print(f"-------------------------------")
print(f"Second chunk : {recurv_splitter[1][:200]}")
print(f"-------------------------------")
print(f"Third chunk : {recurv_splitter[2][:200]}")

Create 4 chunks
First chunk : Machine Learning Basics

Machine learning is a subset of artificial intelligence that enables systems to learn and improve
from experience without being explicitly programmed. It focuses on developing
-------------------------------
Second chunk : on developing computer programs
that can access data and use it to learn for themselves.

Types of Machine Learning:
1. Supervised Learning: Learning with labeled data
2. Unsupervised Learning:
-------------------------------
Third chunk : Learning: Finding patterns in unlabeled data
3. Reinforcement Learning: Learning through rewards and penalties

Applications include image recognition, speech processing, and recommendation


In [90]:
# Create text without a natural break point
simple_text = "This is sentence one and it is quite long. This is sentence two and it is also quite long. This is sentence three which is even longer than the others. This is sentence four. This is sentence five. This is sentence six."

splitter = RecursiveCharacterTextSplitter(
    separators= [" "],
    chunk_size = 80,
    chunk_overlap = 20,
    length_function = len
)

chunks = splitter.split_text(simple_text)

print(f"\n Simple text example - {len(chunks)}, chunks \n")
for i in range(len(chunks)-1):
    print(f"Chunkl {i+1}: {chunks[i]}")
    print(f"Chunkl {i+2}: {chunks[i+1]}")

    print("--")
   
    
    



 Simple text example - 4, chunks 

Chunkl 1: This is sentence one and it is quite long. This is sentence two and it is also
Chunkl 2: two and it is also quite long. This is sentence three which is even longer than
--
Chunkl 2: two and it is also quite long. This is sentence three which is even longer than
Chunkl 3: is even longer than the others. This is sentence four. This is sentence five.
--
Chunkl 3: is even longer than the others. This is sentence four. This is sentence five.
Chunkl 4: is sentence five. This is sentence six.
--


## Method 3: Token text splitter

In [94]:
token_splitter  = TokenTextSplitter(
    chunk_size = 50,
    chunk_overlap = 10
)

token_chunks = token_splitter.split_text(text)
print(f"Create {len(token_chunks)} chunks")
print(f"First chunk: {token_chunks[0][:1000]}")

Create 3 chunks
First chunk: Machine Learning Basics

Machine learning is a subset of artificial intelligence that enables systems to learn and improve
from experience without being explicitly programmed. It focuses on developing computer programs
that can access data and use it to learn for themselves.

Types


In [96]:
print(f"First chunk : {token_chunks[0][:200]}")
print(f"-------------------------------")
print(f"Second chunk : {token_chunks[1][:200]}")
print(f"-------------------------------")
print(f"Third chunk : {token_chunks[2][:200]}")

First chunk : Machine Learning Basics

Machine learning is a subset of artificial intelligence that enables systems to learn and improve
from experience without being explicitly programmed. It focuses on developing
-------------------------------
Second chunk :  use it to learn for themselves.

Types of Machine Learning:
1. Supervised Learning: Learning with labeled data
2. Unsupervised Learning: Finding patterns in unlabeled data
3. Reinforcement Learning: 
-------------------------------
Third chunk : 
3. Reinforcement Learning: Learning through rewards and penalties

Applications include image recognition, speech processing, and recommendation systems



